In [2]:
!pip install xgboost

     |████████████████████████████████| 142.8MB 1.2kB/s eta 0:00:013  |▎                               | 1.2MB 4.1MB/s eta 0:00:35     |▌                               | 2.0MB 4.1MB/s eta 0:00:35     |▋                               | 2.7MB 4.1MB/s eta 0:00:35�                               | 3.3MB 4.1MB/s eta 0:00:34     |▉                               | 3.7MB 4.1MB/s eta 0:00:34     |█▌                              | 6.7MB 3.3MB/s eta 0:00:42     |██▎                             | 10.3MB 2.0MB/s eta 0:01:08     |███▏                            | 14.0MB 2.0MB/s eta 0:01:05     |███▏                            | 14.2MB 1.2MB/s eta 0:01:48     |███▋                            | 16.3MB 1.7MB/s eta 0:01:17     |████                            | 18.2MB 751kB/s eta 0:02:46kB/s eta 0:08:14███▎                        | 32.7MB 108kB/s eta 0:16:55███▋                        | 34.0MB 108kB/s eta 0:16:43     |████████▊                       | 38.7MB 101kB/s eta 0:17:05 101kB/s eta 0:17:03     |█

In [3]:
%reload_ext autoreload
%autoreload 2

import nltk
nltk.download('stopwords')
nltk.download('punkt')


#data_preprocessing
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from bs4 import BeautifulSoup 
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

##import dataset
dataset = pd.read_excel('train.xlsx')

##Fill missing values of Host with that link in the same row
for i in list(dataset[dataset['Host'].isnull() == True]['Host'].index):
    dataset.at[i,'Host'] = dataset.loc[i]['Link']
    
##Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(dataset[dataset['TRANS_CONV_TEXT'].isnull() == True].index):
    dataset.at[i, 'TRANS_CONV_TEXT'] = dataset.loc[i]['Title']
##Converting 'Date(ET)' object to datetime
dataset['Date(ET)'] = pd.to_datetime(dataset['Date(ET)'], infer_datetime_format=True)    

##Converting Datetime to timestamp
dataset['Date(ET)'] = dataset[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

##Dropping time(ET) and time(GMT)
dataset.drop(['Date(ET)', 'Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

##Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(dataset[dataset['Title'].isnull() == True]['Title'].index):
    dataset.at[i,'Title'] = dataset.loc[i]['TRANS_CONV_TEXT']

## TEST Data
test = pd.read_csv('test.csv', encoding = 'utf-8')

##Fill missing values of Host with that link in the same row
for i in list(test[test['Host'].isnull() == True]['Host'].index):
    test.at[i,'Host'] = test.loc[i]['Link']
    
#Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(test[test['TRANS_CONV_TEXT'].isnull() == True].index):
    test.at[i, 'TRANS_CONV_TEXT'] = test.loc[i]['Title']

test.at[441,'Date(ET)'] = test.loc[441, 'Time(ET)']

#Converting 'Date(ET)' object to datetime
test['Date(ET)'] = pd.to_datetime(test['Date(ET)'], infer_datetime_format=True)    

#Converting Datetime to timestamp
test['Date(ET)'] = test[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

#Dropping time(ET) and time(GMT)
test.drop(['Date(ET)','Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

#Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(test[test['Title'].isnull() == True]['Title'].index):
    test.at[i,'Title'] = test.loc[i]['TRANS_CONV_TEXT']

index = test['Index']
index = list(index)

test.drop(['Index'], axis = 1, inplace = True)

#feature_selection
y = dataset['Patient_Tag'].tolist()

dataset['Story'] = 'a'


for i in range(len(dataset)):
    dataset.at[i, 'Story'] = dataset['Source'][i] + ' ' + dataset['Host'][i] + ' ' + str(dataset['Link'][i]) + ' ' + dataset['Title'][i] + ' ' + dataset['TRANS_CONV_TEXT'][i]

dataset.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT'], axis = 1, inplace = True)


test['Story'] = 'a'


for i in range(len(test)):
    test.at[i, 'Story'] = test['Source'][i] + ' ' + test['Host'][i] + ' ' + str(test['Link'][i]) + ' ' + test['Title'][i] + ' ' + test['TRANS_CONV_TEXT'][i]

test.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT', 'Unnamed: 9'], axis = 1, inplace = True)

replace_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def preprocess(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_space.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = bad_symbols.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text 

dataset['Story'] = dataset['Story'].apply(preprocess)
test['Story'] = test['Story'].apply(preprocess)

data_train_list = dataset['Story'].tolist()
data_test_list = test['Story'].tolist()

all_data = data_train_list + data_test_list



tt = WordPunctTokenizer()
count_vect=CountVectorizer(tokenizer = tt.tokenize ,stop_words ='english', ngram_range = (1,3))
count_vect.fit(all_data)
train_countvect = count_vect.transform(dataset.Story)
test_countvect = count_vect.transform(test.Story)

## Feature Selection
#Dimensionality reduction using LSI
svd=TruncatedSVD(n_components=50,n_iter=10,random_state=42)
svd.fit(train_countvect)
train_countvect_LSI=svd.transform(train_countvect)

svd=TruncatedSVD(n_components=50,n_iter=10,random_state=42)
svd.fit(test_countvect)
test_countvect_LSI=svd.transform(test_countvect)

#Feature extraction using TFxIDF
tf_idf = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1.0,smooth_idf=1.0,sublinear_tf=1.0,
            stop_words = 'english')
tf_idf.fit(all_data)
train_tf= tf_idf.transform(dataset.Story)
test_tf = tf_idf.transform(test.Story)

## Feature Selection
#Dimensionality reduction using LSI
svd=TruncatedSVD(n_components=50,n_iter=10,random_state=42)
svd.fit(train_tf)
train_tf_LSI=svd.transform(train_tf)

svd=TruncatedSVD(n_components=50,n_iter=10,random_state=42)
svd.fit(test_tf)
test_tf_LSI=svd.transform(test_tf)






[nltk_data] Downloading package stopwords to /home/nbuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/nbuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Adaboost

In [1]:
'''
# defining parameter range 
param_grid = {'n_estimators': np.arange(10,300,25),
     'learning_rate': [0.01, 0.05, 0.1, 1]
             }  
cv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  
grid = GridSearchCV(AdaBoostClassifier(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) 

# fitting the model for grid search 
grid.fit(train_tf_LSI, y) 

# print best parameter after tuning 
print(grid.best_params_) 

# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_)
'''

"\n# defining parameter range \nparam_grid = {'n_estimators': np.arange(10,300,25),\n     'learning_rate': [0.01, 0.05, 0.1, 1]\n             }  \ncv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  \ngrid = GridSearchCV(AdaBoostClassifier(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) \n\n# fitting the model for grid search \ngrid.fit(train_tf_LSI, y) \n\n# print best parameter after tuning \nprint(grid.best_params_) \n\n# print how our model looks after hyper-parameter tuning \nprint(grid.best_estimator_)\n"

In [7]:
best_params = {'n_estimators': 200 , 'learning_rate': 1 }
ada_clf = AdaBoostClassifier(**best_params)
ada_clf.fit(train_tf_LSI, y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
          n_estimators=200, random_state=None)

### GradientBoosting

In [4]:
'''
# defining parameter range 
param_grid = {'n_estimators': [10,25,50,75,100],
     'learning_rate': [0.01, 0.05, 0.1, 1]
             }  
cv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  
grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) 

# fitting the model for grid search 
grid.fit(train_tf_LSI, y) 

# print best parameter after tuning 
print(grid.best_params_) 

# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_)
'''

"\n# defining parameter range \nparam_grid = {'n_estimators': [10,25,50,75,100],\n     'learning_rate': [0.01, 0.05, 0.1, 1],\n              'max_depth' : np.arange(1,10)\n             }  \ncv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  \ngrid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) \n\n# fitting the model for grid search \ngrid.fit(train_tf_LSI, y) \n\n# print best parameter after tuning \nprint(grid.best_params_) \n\n# print how our model looks after hyper-parameter tuning \nprint(grid.best_estimator_)\n"

In [8]:
best_params = {'n_estimators': 200, 'learning_rate': 1}
grad_clf = GradientBoostingClassifier(**best_params)
grad_clf.fit(train_tf_LSI, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

### XGBoost


In [9]:
'''
# defining parameter range 
param_grid = {'reg_alpha': np.arange(0,5),
              'reg_lambda': np.arange(0,5),
     'learning_rate': [0.01, 0.05, 0.1, 1],
              'max_depth' = np.arange(1,10)
             }  
cv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  
grid = GridSearchCV(XBGClassifier(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) 

# fitting the model for grid search 
grid.fit(train_tf_LSI, y) 

# print best parameter after tuning 
print(grid.best_params_) 

# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_)
'''

"\n# defining parameter range \nparam_grid = {'reg_alpha': np.arange(0,5),\n              'reg_lambda': np.arange(0,5),\n     'learning_rate': [0.01, 0.05, 0.1, 1],\n              'max_depth' = np.arange(1,10)\n             }  \ncv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  \ngrid = GridSearchCV(XBGClassifier(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) \n\n# fitting the model for grid search \ngrid.fit(train_tf_LSI, y) \n\n# print best parameter after tuning \nprint(grid.best_params_) \n\n# print how our model looks after hyper-parameter tuning \nprint(grid.best_estimator_)\n"

In [12]:

best_params = {'reg_alpha': 1, 'reg_lambda': 1, 'learning_rate': 1}
xgb_clf = XGBClassifier(**best_params)
xgb_clf.fit(train_tf_LSI, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=1,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

### RandomForestClassifier

In [ ]:
# defining parameter range 
param_grid = {'n_estimators': [250, 500, 750, 1000, 1500, 2000],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [25, 50, 75, 100]
             }  
cv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  
grid = GridSearchCV(RandomForestClassifier(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) 

# fitting the model for grid search 
grid.fit(train_tf_LSI, y) 

# print best parameter after tuning 
print(grid.best_params_) 

# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_)

In [ ]:
best_params = {'n_estimators': ,
               'max_features': ,
               'max_depth': }
rf_clf = RandomForestClassifier(**best_params)
rf_clf.fit(train_tf_LSI, y)

### SVC

In [13]:
'''
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],
             'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}  
cv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  
grid = GridSearchCV(SVC(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) 

# fitting the model for grid search 
grid.fit(train_tf_LSI, y) 

# print best parameter after tuning 
print(grid.best_params_) 

# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 
'''

"\n# defining parameter range \nparam_grid = {'C': [0.1, 1, 10, 100, 1000],\n             'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}  \ncv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  \ngrid = GridSearchCV(SVC(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) \n\n# fitting the model for grid search \ngrid.fit(train_tf_LSI, y) \n\n# print best parameter after tuning \nprint(grid.best_params_) \n\n# print how our model looks after hyper-parameter tuning \nprint(grid.best_estimator_) \n"

In [14]:
best_params = {'kernel': 'linear', 'C': 1000}
svc_tf = SVC(**best_params, class_weight='balanced', probability=True)
svc_tf.fit(train_tf_LSI, y)

SVC(C=1000, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

### Logistic Regression for CountVect

In [10]:
'''
from sklearn.linear_model import LogisticRegression
# defining parameter range 
param_grid = {'C': [0.1, 1, 5, 10,50, 100],
             'max_iter': [100,200],
             'multi_class': ['auto', 'ovr'],
              'penalty': ['l1', 'l2']
             }  
cv = KFold(train_countvect_LSI.shape[0], shuffle=True, random_state=0)  
grid = GridSearchCV(LogisticRegression(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) 

# fitting the model for grid search 
grid.fit(train_countvect_LSI, y) 

# print best parameter after tuning 
print(grid.best_params_) 

# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 
'''

"\nfrom sklearn.linear_model import LogisticRegression\n# defining parameter range \nparam_grid = {'C': [0.1, 1, 5, 10,50, 100],\n             'max_iter': [100,200],\n             'multi_class': ['auto', 'ovr'],\n              'penalty': ['l1', 'l2']\n             }  \ncv = KFold(train_countvect_LSI.shape[0], shuffle=True, random_state=0)  \ngrid = GridSearchCV(LogisticRegression(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) \n\n# fitting the model for grid search \ngrid.fit(train_countvect_LSI, y) \n\n# print best parameter after tuning \nprint(grid.best_params_) \n\n# print how our model looks after hyper-parameter tuning \nprint(grid.best_estimator_) \n"

In [15]:
best_params = {'C': 1,'max_iter': 100, 'multi_class': 'auto','penalty': 'l1'}
lr = LogisticRegression(**best_params, class_weight='balanced')
lr.fit(train_countvect_LSI, y)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='auto', n_jobs=None, penalty='l1', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

### Logistic Regression for TFIDF

In [12]:
'''
# defining parameter range 
param_grid = {'C': [1, 5, 10, 100, 250, 500, 1000],
             'max_iter': [50,100,200],
             'multi_class': ['auto', 'ovr', 'multinomial'],
              'penalty': ['l1', 'l2', 'elasticnet']
             }  
cv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  
grid = GridSearchCV(LogisticRegression(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) 

# fitting the model for grid search 
grid.fit(train_tf_LSI, y) 

# print best parameter after tuning 
print(grid.best_params_) 

# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 
'''

"\n# defining parameter range \nparam_grid = {'C': [1, 5, 10, 100, 250, 500, 1000],\n             'max_iter': [50,100,200],\n             'multi_class': ['auto', 'ovr', 'multinomial'],\n              'penalty': ['l1', 'l2', 'elasticnet']\n             }  \ncv = KFold(train_tf_LSI.shape[0], shuffle=True, random_state=0)  \ngrid = GridSearchCV(LogisticRegression(), param_grid, cv = cv, n_jobs = -1, refit = True, verbose = 3) \n\n# fitting the model for grid search \ngrid.fit(train_tf_LSI, y) \n\n# print best parameter after tuning \nprint(grid.best_params_) \n\n# print how our model looks after hyper-parameter tuning \nprint(grid.best_estimator_) \n"

In [16]:
best_params = {'C': 250, 'max_iter': 100, 'penalty': 'l2'}
lr_tf = LogisticRegression(**best_params, class_weight='balanced')
lr_tf.fit(train_tf_LSI, y)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=250, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

### Ensembling

In [ ]:
from scipy.stats.mstats import mode

pred1 = ada_clf.predict_proba(test_tf_LSI)
pred2 = grad_clf.predict_proba(test_tf_LSI)
pred3 = xgb_clf.predict_proba(test_tf_LSI)
pred4 = rf_clf.predict_proba(test_tf_LSI)
pred5 = svc_tf.predict_proba(test_tf_LSI)
pred6 = lr.predict_proba(test_countvect_LSI)
pred7 = lr_tf.predict_proba(test_tf_LSI)



test_pred_prob = np.mean([pred1, pred2, pred3, pred4, pred5, pred6, pred7], axis=0)
pred = np.argmax(test_pred_prob, axis=1)


In [ ]:
submission = pd.DataFrame()
submission['Index'] = index
submission['SECTION'] = pred.astype(int)
submission.to_csv("ml_prob.csv", index=None)
submission.head()

### LSTM

In [ ]:
%reload_ext autoreload
%autoreload 2

import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
#data_preprocessing
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from bs4 import BeautifulSoup 
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
#from xgboost import XGBClassifier

##import dataset
dataset = pd.read_excel('train.xlsx')

##Fill missing values of Host with that link in the same row
for i in list(dataset[dataset['Host'].isnull() == True]['Host'].index):
    dataset.at[i,'Host'] = dataset.loc[i]['Link']
    
##Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(dataset[dataset['TRANS_CONV_TEXT'].isnull() == True].index):
    dataset.at[i, 'TRANS_CONV_TEXT'] = dataset.loc[i]['Title']
##Converting 'Date(ET)' object to datetime
dataset['Date(ET)'] = pd.to_datetime(dataset['Date(ET)'], infer_datetime_format=True)    

##Converting Datetime to timestamp
dataset['Date(ET)'] = dataset[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

##Dropping time(ET) and time(GMT)
dataset.drop(['Date(ET)', 'Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

##Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(dataset[dataset['Title'].isnull() == True]['Title'].index):
    dataset.at[i,'Title'] = dataset.loc[i]['TRANS_CONV_TEXT']

## TEST Data
test = pd.read_csv('test.csv', encoding = 'utf-8')

##Fill missing values of Host with that link in the same row
for i in list(test[test['Host'].isnull() == True]['Host'].index):
    test.at[i,'Host'] = test.loc[i]['Link']
    
#Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(test[test['TRANS_CONV_TEXT'].isnull() == True].index):
    test.at[i, 'TRANS_CONV_TEXT'] = test.loc[i]['Title']

test.at[441,'Date(ET)'] = test.loc[441, 'Time(ET)']

#Converting 'Date(ET)' object to datetime
test['Date(ET)'] = pd.to_datetime(test['Date(ET)'], infer_datetime_format=True)    

#Converting Datetime to timestamp
test['Date(ET)'] = test[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

#Dropping time(ET) and time(GMT)
test.drop(['Date(ET)','Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

#Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(test[test['Title'].isnull() == True]['Title'].index):
    test.at[i,'Title'] = test.loc[i]['TRANS_CONV_TEXT']

index = test['Index']
index = list(index)

test.drop(['Index'], axis = 1, inplace = True)

#feature_selection
y = dataset['Patient_Tag'].tolist()

dataset['Story'] = 'a'


for i in range(len(dataset)):
    dataset.at[i, 'Story'] = dataset['Source'][i] + ' ' + dataset['Host'][i] + ' ' + str(dataset['Link'][i]) + ' ' + dataset['Title'][i] + ' ' + dataset['TRANS_CONV_TEXT'][i]

dataset.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT'], axis = 1, inplace = True)


test['Story'] = 'a'


for i in range(len(test)):
    test.at[i, 'Story'] = test['Source'][i] + ' ' + test['Host'][i] + ' ' + str(test['Link'][i]) + ' ' + test['Title'][i] + ' ' + test['TRANS_CONV_TEXT'][i]

test.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT', 'Unnamed: 9'], axis = 1, inplace = True)

replace_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def preprocess(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text 

dataset['Story'] = dataset['Story'].map(lambda x: preprocess(x))
test['Story'] = test['Story'].map(lambda x: preprocess(x))

data_train_list = dataset['Story'].tolist()
data_test_list = test['Story'].tolist()

all_data = data_train_list + data_test_list




#Part 1
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import string

from sklearn.manifold import TSNE


### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(dataset['Story'])
sequences = tokenizer.texts_to_sequences(dataset['Story'])
data = pad_sequences(sequences, maxlen=50)

#test
sequences_test = tokenizer.texts_to_sequences(test['Story'])
x_test = pad_sequences(sequences_test, maxlen = 50)


## Network architecture
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model.fit(data, dataset['Patient_Tag'], validation_split=0.3, epochs=25)



y_pred = model.predict([x_test], batch_size=32, verbose=0)
y_test = []
for i in range(len(y_pred)):
  if y_pred[i][0] > 0.7:
    y_test.append(1)
  else:
    y_test.append(0)

submission = pd.DataFrame()
submission['Index'] = index
submission['Patient_Tag'] = y_test
submission.to_csv("lstm_.csv", index=None)
submission.head()



# Part 2: LSTM with CNN
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from sklearn.manifold import TSNE


def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
    return model_conv

model_conv = create_conv_model()
model_conv.fit(data, dataset['Patient_Tag'], validation_split=0.3, epochs = 15)



y_pred = model_conv.predict([x_test], batch_size=32, verbose=0)
y_test = []
for i in range(len(y_pred)):
  if y_pred[i][0] > 0.7:
    y_test.append(1)
  else:
    y_test.append(0)

submission = pd.DataFrame()
submission['Index'] = index
submission['Patient_Tag'] = y_test
submission.to_csv("lstm_cnn.csv", index=None)
submission.head()



# Part 3: LSTM+CNN+Glove Embedding

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE

# Extract word embeddings from Glove
embeddings_index = dict()
f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


# Create a weight matrix
embedding_matrix = np.zeros((vocabulary_size, 50))
for word, index_ in tokenizer.word_index.items():
    if index_ > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index_] = embedding_vector

## create model
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 50, input_length=50, weights=[embedding_matrix], trainable=True))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit train data
model_glove.fit(data, dataset['Patient_Tag'], validation_split=0.4, epochs = 25)


y_pred = model_glove.predict([x_test], batch_size=32, verbose=0)
y_test = []
for i in range(len(y_pred)):
  if y_pred[i][0] > 0.7:
    y_test.append(1)
  else:
    y_test.append(0)

submission = pd.DataFrame()
submission['Index'] = index
submission['Patient_Tag'] = y_test
submission.to_csv("lstm_cnn_glove.csv", index=None)
submission.head()






### Fastai

In [ ]:
!pip install fastai

%reload_ext autoreload
%autoreload 2

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

#data_preprocessing
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import statsmodels.api as sm
from scipy.stats import randint as sp_randint
from time import time

from textblob import TextBlob
from sklearn.feature_extraction import text
from nltk.tokenize import TweetTokenizer

from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import GridSearchCV
from bs4 import BeautifulSoup 
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_curve

##import dataset
dataset = pd.read_excel('train.xlsx')

##Fill missing values of Host with that link in the same row
for i in list(dataset[dataset['Host'].isnull() == True]['Host'].index):
    dataset.at[i,'Host'] = dataset.loc[i]['Link']
    
##Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(dataset[dataset['TRANS_CONV_TEXT'].isnull() == True].index):
    dataset.at[i, 'TRANS_CONV_TEXT'] = dataset.loc[i]['Title']
##Converting 'Date(ET)' object to datetime
dataset['Date(ET)'] = pd.to_datetime(dataset['Date(ET)'], infer_datetime_format=True)    

##Converting Datetime to timestamp
dataset['Date(ET)'] = dataset[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

##Dropping time(ET) and time(GMT)
dataset.drop(['Date(ET)', 'Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

##Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(dataset[dataset['Title'].isnull() == True]['Title'].index):
    dataset.at[i,'Title'] = dataset.loc[i]['TRANS_CONV_TEXT']

## TEST Data
test = pd.read_csv('test.csv', encoding = 'utf-8')

##Fill missing values of Host with that link in the same row
for i in list(test[test['Host'].isnull() == True]['Host'].index):
    test.at[i,'Host'] = test.loc[i]['Link']
    
#Filling missing value of TRANS_CONV_TEXT with that of title
for i in list(test[test['TRANS_CONV_TEXT'].isnull() == True].index):
    test.at[i, 'TRANS_CONV_TEXT'] = test.loc[i]['Title']

test.at[441,'Date(ET)'] = test.loc[441, 'Time(ET)']

#Converting 'Date(ET)' object to datetime
test['Date(ET)'] = pd.to_datetime(test['Date(ET)'], infer_datetime_format=True)    

#Converting Datetime to timestamp
test['Date(ET)'] = test[['Date(ET)']].apply(lambda x: x[0].timestamp(), axis = 1).astype(int)

#Dropping time(ET) and time(GMT)
test.drop(['Date(ET)','Time(ET)', 'time(GMT)'], axis = 1, inplace = True)

#Fill missing values of Title with that of TRANS_CONV_TEXT in the same row
for i in list(test[test['Title'].isnull() == True]['Title'].index):
    test.at[i,'Title'] = test.loc[i]['TRANS_CONV_TEXT']

index = test['Index']
index = list(index)

test.drop(['Index'], axis = 1, inplace = True)

#feature_selection
y = dataset['Patient_Tag'].tolist()

dataset['Story'] = 'a'


for i in range(len(dataset)):
    dataset.at[i, 'Story'] = dataset['Source'][i] + ' ' + dataset['Host'][i] + ' ' + str(dataset['Link'][i]) + ' ' + dataset['Title'][i] + ' ' + dataset['TRANS_CONV_TEXT'][i]

dataset.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT'], axis = 1, inplace = True)


test['Story'] = 'a'


for i in range(len(test)):
    test.at[i, 'Story'] = test['Source'][i] + ' ' + test['Host'][i] + ' ' + str(test['Link'][i]) + ' ' + test['Title'][i] + ' ' + test['TRANS_CONV_TEXT'][i]

test.drop(['Source', 'Host', 'Link', 'Title', 'TRANS_CONV_TEXT', 'Unnamed: 9'], axis = 1, inplace = True)

replace_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def preprocess(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_space.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = bad_symbols.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text 

dataset['Story'] = dataset['Story'].apply(preprocess)
test['Story'] = test['Story'].apply(preprocess)

data_train_list = dataset['Story'].tolist()
data_test_list = test['Story'].tolist()

all_data = data_train_list + data_test_list

print("size of test data as compared to train data {:0.2f}%".format(len(test)/len(dataset)*100))

#Returning average word length of phrases for each dataset
print('Average word length of phrases in train is {0:.0f}.'.format(np.mean(dataset['Story'].apply(lambda x: len(x.split())))))
print('Average word length of phrases in test is {0:.0f}.'.format(np.mean(test['Story'].apply(lambda x: len(x.split())))))

#Graph
fig, ax = plt.subplots(1,1, dpi = 100, figsize = (10,5))
data_ = pd.read_excel('train.xlsx')
patient_tags_labels = data_['Patient_Tag'].value_counts().index
patient_tags_count = data_['Patient_Tag'].value_counts()

#Plotting
sns.barplot(x = patient_tags_labels, y = patient_tags_count)

ax.set_ylabel('Count')
ax.set_xlabel('Patient Tag Label')
ax.set_xticklabels(patient_tags_labels)


from fastai.text import *

data = (TextList.from_df(dataset, cols = 'Story').split_by_rand_pct(0.2).label_for_lm().databunch(bs = 48))
data.show_batch()

learn = language_model_learner(data, AWD_LSTM, drop_mult = 0.3)
learn.fit_one_cycle(4)
learn.save('stage-1')
learn.unfreeze()
learn.fit_one_cycle(1)
learn.load('stage-1')

learn.lr_find()
learn.recorder.plot()

learn.unfreeze()
learn.fit_one_cycle(3)
learn.save_encoder('fine_tuned_enc')
learn.load_encoder('fine_tuned_enc')

test_datalist = TextList.from_df(test, cols = 'Story', vocab = data.vocab)

data_clas = (TextList.from_df(dataset, cols = 'Story', vocab = data.vocab).split_by_rand_pct(0.2)
            .label_from_df(cols = 'Patient_Tag')
            .add_test(test_datalist)
            .databunch(bs = 32))

data_clas.show_batch()

learn_classifier = text_classifier_learner(data_clas, AWD_LSTM, drop_mult = 0.3)
learn_classifier.load_encoder('fine_tuned_enc')
learn_classifier.freeze()

learn_classifier.lr_find()
learn_classifier.recorder.plot()
learn_classifier.fit_one_cycle(1, 2e-2, moms = (0.8, 0.7))
learn_classifier.save('first')
learn_classifier.load('first')

learn_classifier.freeze_to(-2)
learn_classifier.fit_one_cycle(1, slice(1e-2/(2.6**4), 1e-2), moms = (0.8, 0.7))
learn_classifier.save('second')
learn_classifier.load('second')

learn_classifier.freeze_to(-3)
learn_classifier.fit_one_cycle(1, slice(5e-3/(2.6**4), 5e-3), moms = (0.8, 0.7))

learn_classifier.save('third')
learn_classifier.load('third')

learn_classifier.unfreeze()
learn_classifier.fit_one_cycle(3, slice(1e-3/(2.6**4), 1e-3), moms = (0.8,0.7))

preds, target = learn_classifier.get_preds(DatasetType.Test, ordered = True)
labels = np.argmax(preds, axis = 1)

learn_classifier.show_results()

np.save("fastai_prob.npy", preds)

submission = pd.DataFrame()
submission['Index'] = index
submission['Patient_Tag'] = labels
submission.to_csv("fastai_model.csv", index=None)
submission.head()



### Final Ensemble

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import mode

pred1 = pd.read_csv("ml_prob.csv", index_col=None)
pred2 = np.argmax(np.load('fastai_prob.npy'), axis=1)
pred3 = pd.read_csv("lstm_.csv", index_col=None)
pred4 = pd.read_csv("lstm_cnn.csv", index_col=None)
pred5 = pd.read_csv("lstm_cnn_glove.csv", index_col=None)


final_ensemble = mode((pred1['Patient_Tag'], pred2, pred3['Patient_Tag'], pred4['Patient_Tag'], pred5['Patient_Tag']))[0][0]
submission = pd.DataFrame()
submission['Index'] = pred1['Index']
submission['Patient_Tag'] = final_ensemble
submission.to_csv('ensemble_final.csv', index=None)
print(submission['Patient_Tag'].value_counts())

